In [ ]:
import uproot3 as uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
## Import files with residuals
nolaser = "../output/residuals/run8552_cosmics_timing_residuals_20230725_noLaser.csv"
laser =  "../output/residuals/run8552_cosmics_timing_residuals_20230725.csv"
cosmics = "../output/residuals/run8552_cosmics_timing_residuals_20230725_CORRw8461.csv"
#cosmics = "../output/residuals/run8437_cosmics_timing_residuals_20230715_CORRw8413.csv"

In [ ]:
dfnolaser = pd.read_csv(nolaser).set_index(["channel_id"])
dflaser = pd.read_csv(laser).set_index(["channel_id"])
dfcosmics = pd.read_csv(cosmics).set_index(["channel_id"])

In [ ]:
len(dfcosmics)
dfcosmics.head()

## Comparison between before/after corrections

In [ ]:
## Plotting residuals distribution with/without corrections
fig = plt.figure(figsize=(10,5),dpi=100)

rmin=-10
rmax=10
r=(rmin,rmax)
s=0.5
b=int((rmax-rmin)/s)

l_nolaser = "Base sample\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(dfnolaser.mean_residual_ns),np.std(dfnolaser.mean_residual_ns))
l_laser = "Laser only\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(dflaser.mean_residual_ns),np.std(dflaser.mean_residual_ns))
l_cosmics = "Laser + cosmics\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(dfcosmics.mean_residual_ns),np.std(dfcosmics.mean_residual_ns))

plt.hist(dfnolaser.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l_nolaser)
plt.hist(dflaser.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l_laser)
plt.hist(dfcosmics.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l_cosmics)

plt.xlabel("Time residuals [ns]")
plt.ylabel("Number of PMTs")
plt.title("ICARUS Run 1 - PMT Timing Equalization")
plt.legend()
plt.grid(linestyle="dashed",alpha=0.5)
#plt.savefig("../figs/run8552_validation_CORRw8461",dpi=100)
plt.show()

## Comparison between laser/cosmics residuals

In [ ]:
LASERCORR="../inputs/laser_time_corrections_20220408.csv"
COSMICSCORR="../output/residuals/run8461_cosmics_timing_residuals_20230725.csv"
VOLTAGE="../inputs/pmt_voltage_20220408.csv"
lasercorr = pd.read_csv(LASERCORR).set_index(["channel_id"])
voltage = pd.read_csv(VOLTAGE).set_index(["channel_id"])
thisdfg = pd.read_csv(COSMICSCORR).set_index(["channel_id"])
thisdfg = thisdfg.join( lasercorr[["t_signal", "A", "B", "C"]], on="channel_id" ) 
thisdfg = thisdfg.join( voltage, on="channel_id" ) 
thisdfg["laser_res"] = thisdfg.apply( lambda x : x.t_signal-(x.A+x.B*np.power(x.voltage, x.C) ) , axis=1)

In [ ]:
# Verify potential presence of biases 
fig, ax = plt.subplots(1,2, figsize=(16, 5))
ax[0].errorbar( x=thisdfg.y, y=thisdfg.mean_residual_ns, yerr=thisdfg.emean_ns, marker='.', elinewidth=2.0, lw=0 )
ax[1].errorbar( x=thisdfg.y, y=thisdfg.laser_res, yerr=0, marker='.', elinewidth=2.0, lw=0 )
ax[1].set_ylabel("Laser Residuals [ns]", fontsize=16)
ax[0].set_ylabel("Cosmics Residuals [ns]", fontsize=16)
ax[1].set_xlabel("Y position [cm]", fontsize=16)
ax[0].set_xlabel("Y position [cm]", fontsize=16)
plt.tight_layout()
ax[0].grid(alpha=0.5,linestyle="dashed")
ax[1].grid(alpha=0.5,linestyle="dashed")
plt.savefig("../figs/run8552_validation_laserVScomics_Y.png",dpi=100)
plt.show()

fig, ax = plt.subplots(1,2, figsize=(16, 5))
ax[0].errorbar( x=thisdfg.z, y=thisdfg.mean_residual_ns, yerr=thisdfg.emean_ns, marker='.', elinewidth=2.0, lw=0 )
ax[1].errorbar( x=thisdfg.z, y=thisdfg.laser_res, yerr=0, marker='.', elinewidth=2.0, lw=0 )
ax[1].set_ylabel("Laser Residuals [ns]", fontsize=16)
ax[0].set_ylabel("Cosmics Residuals [ns]", fontsize=16)
ax[1].set_xlabel("Z position [cm]", fontsize=16)
ax[0].set_xlabel("Z position [cm]", fontsize=16)
plt.tight_layout()
ax[0].grid(alpha=0.5,linestyle="dashed")
ax[1].grid(alpha=0.5,linestyle="dashed")
plt.savefig("../figs/run8552_validation_laserVScomics_Z.png",dpi=100)
plt.show()

# plot them togheter
fig, ax = plt.subplots(1,1, figsize=(8, 5))
ax.errorbar( x=thisdfg.mean_residual_ns, y=thisdfg.laser_res, yerr=0, marker='o', elinewidth=2.0, lw=0, alpha=0.5 )
ax.set_ylabel("Laser Residuals [ns]", fontsize=16)
ax.set_xlabel("Cosmics Residuals [ns]", fontsize=16)

plt.tight_layout()
plt.grid(alpha=0.5,linestyle="dashed")
plt.savefig("../figs/run8552_validation_laserVScomics_2Dscatter.png",dpi=100)
plt.show()

In [ ]:
thisdfg[thisdfg.mean_residual_ns<-2].head()

## Compare statistics

In [ ]:
f1 = "../output/residuals/run8460_cosmics_timing_residuals_20230725.csv"
f2 = "../output/residuals/run8461_cosmics_timing_residuals_20230725.csv"
f3 = "../output/residuals/run8462_cosmics_timing_residuals_20230725.csv"
f4 = "../output/residuals/run8552_cosmics_timing_residuals_20230725.csv"

df1 = pd.read_csv(f1)
df2 = pd.read_csv(f2)
df3 = pd.read_csv(f3)
df4 = pd.read_csv(f4)

fig = plt.figure(figsize=(10,5),dpi=100)

l1 = "Run 8460\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(df1.mean_residual_ns),np.std(df1.mean_residual_ns))
l2 = "Run 8461\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(df2.mean_residual_ns),np.std(df2.mean_residual_ns))
l3 = "Run 8462\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(df3.mean_residual_ns),np.std(df3.mean_residual_ns))
l4 = "Run 8552\nMean: {:.2f} ns\nStd: {:.2f} ns".format(np.mean(df4.mean_residual_ns),np.std(df4.mean_residual_ns))

plt.hist(df1.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l1)
plt.hist(df2.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l2)
plt.hist(df3.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l3)
plt.hist(df4.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=l4)
plt.xlabel("Time residuals [ns]")
plt.ylabel("Number of PMTs")
plt.legend()
plt.grid(linestyle="dashed",alpha=0.5)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5),dpi=100)

l1 = "Run 8460"
l2 = "Run 8461"
l3 = "Run 8462"
l4 = "Run 8552"
plt.hist(df1.entries, linewidth=2, histtype="step", label=l1)
plt.hist(df2.entries, linewidth=2, histtype="step", label=l2)
plt.hist(df3.entries, linewidth=2, histtype="step", label=l3)
plt.hist(df4.entries, linewidth=2, histtype="step", label=l4)
plt.xlabel("Entries per channel")
plt.ylabel("Number of PMTs")
plt.legend()
plt.grid(linestyle="dashed",alpha=0.5)
plt.show()

## Test old file cuts from Andrea

In [ ]:
f = "../output/run8413_matched_light_tracks_Andrea.txt"
df = pd.read_csv(f)
df.head()

In [ ]:
plt.hist( df.flashTime/1000 - df.trackTime/1000, histtype="step")
plt.show()

In [ ]:
plt.hist( df.flashLightZ - df.trackChargeZ, histtype = "step")
plt.show()